# Fault Filtering with GeoIO

This notebook will show how to load polygon into nDI, rotate the color and filter with a polar widget. 

Author: Jie Hou Modified Date: 11/30/2020

### import necessary packages

In [1]:
from geoio.geoio import GeoIoFaultCollection, GeoIoDataLoadAndDisplay, GeoIoDladViewerType, GeoIoDladFeatureColor

import pandas as pd

import numpy as np

import random

import plotly.graph_objects as go

import math

import os

In [2]:
help(GeoIoDataLoadAndDisplay)

Help on class GeoIoDataLoadAndDisplay in module geoio.geoio:

class GeoIoDataLoadAndDisplay(builtins.object)
 |  GeoIoDataLoadAndDisplay(*args)
 |  
 |  GeoIoDataLoadAndDisplay wraps the methods of nDI DataLoadAndDisplay class
 |  
 |     The constructor takes an input data file and coordinate system as the arguments.
 |     The supported data types are volume (.vt, .vds, .amp), horizon collection (.int), 
 |     fault collection (.flt), well, earth model (.bin), mapl collection (.saf, .vs)
 |     trimesh (.ts) and polygon (.ply)
 |  
 |     To instantiate a GeoIoDataLoadAndDisplay, pass the data filepath 
 |     and a coordinate system type(optional) to the constructor
 |  
 |        .. function:: GeoIoDataLoadAndDisplay(filepath, GeoIoDladCoordinateSystem)
 |  
 |           :param str filename: input data file
 |           :param GeoIoDladCoordinateSystem coordinate system: (optional) the coordinate systerm. The default is WORLDXY.
 |           :raises ValueError: Not able to find th

def a function to calculate strike and dip for a fault

In [2]:
def calc_strikedip(pts):
    ptA, ptB, ptC = pts[0], pts[1], pts[2]
    x1, y1, z1 = float(ptA[0]), float(ptA[1]), float(ptA[2])
    x2, y2, z2 = float(ptB[0]), float(ptB[1]), float(ptB[2])
    x3, y3, z3 = float(ptC[0]), float(ptC[1]), float(ptC[2])

    u1 = float(((y1 - y2) * (z3 - z2) - (y3 - y2) * (z1 - z2)))
    u2 = float((-((x1 - x2) * (z3 - z2) - (x3 - x2) * (z1 - z2))))
    u3 = float(((x1 - x2) * (y3 - y2) - (x3 - x2) * (y1 - y2)))

    if u3 < 0:
        easting = u2
    else:
        easting = -u2

    if u3 > 0:
        northing = u1
    else:
        northing = -u1

    if easting >= 0:
        partA_strike = math.pow(easting, 2) + math.pow(northing, 2)
        strike = math.degrees(math.acos(northing / math.sqrt(partA_strike)))
    else:
        partA_strike = northing / math.sqrt(math.pow(easting, 2) + math.pow(northing, 2))
        strike = math.degrees(2 * math.pi - math.acos(partA_strike))

    # determine dip
    # print(strike, 'strike')
    part1_dip = math.sqrt(math.pow(u2, 2) + math.pow(u1, 2))
    part2_dip = math.sqrt(math.pow(u1,2) + math.pow(u2,2) + math.pow(u3,2))
    dip = math.degrees(math.asin(part1_dip / part2_dip))

    return [strike, dip]

In [10]:
FAULT_FILE = '/glb/data/siep_rii/sgsdata/usr/usjhs3/Demo/DLAD/GW_Frio_faults.flt'

In [4]:
fault_collection = GeoIoFaultCollection(FAULT_FILE)

In [5]:
fault_names = fault_collection.get_names()

In [6]:
faults = pd.DataFrame(columns=['fault_name','strike','dip','length'])

Compute strike and dipe for each fault in the fault collection

In [9]:
for fault_name in fault_names:
    if ((not fault_name.startswith('DELETE')) and (not fault_name.startswith('UNASSIGN'))):
        fault = fault_collection.get_fault(fault_name)
        fault_segments = fault.get_segments()
        fault_data = pd.DataFrame(columns=['x','y','z'])
        for segment in fault_segments:
            fault_data = fault_data.append(pd.DataFrame(segment.get_points(),columns=['x','y','z']))
        count = 0
        strike_sum = dip_sum = 0
        while count < 10:
            try:
                if (fault_data.shape[0]>3):
                    p1, p2, p3 = np.sort(random.sample(range(0,fault_data.shape[0]),3))
                    strike, dip = calc_strikedip(fault_data.iloc[[p1,p2,p3]].to_numpy())
                    strike_sum += strike
                    dip_sum += dip
                    count +=1
            except:
                print("error")
                pass
        strike = strike_sum/count
        dip = dip_sum/count
        length = (fault_data.z.max() - fault_data.z.min())/math.sin(dip/180*math.pi)
        faults = faults.append({'fault_name':fault_name,'strike':strike,'dip':dip,'length':length},ignore_index=True)

Launch nDI Volume

In [29]:
os.system("/apps/sss/share/nDIVolume&")

0

### Load the fault file and turn on in nDI

In [30]:
fault_dlad = GeoIoDataLoadAndDisplay(FAULT_FILE)

In [31]:
fault_dlad.load()

In [32]:
fault_dlad.turn_on_items()

## Rotate the color of the faults

In [33]:
colors = [item for item in dir(GeoIoDladFeatureColor) if not item.startswith('__') and item != 'None']

In [34]:
for fault_name in fault_names:
    fault_dlad.turn_on_items(fault_name,GeoIoDladViewerType.VOLUMEVIEWER,getattr(GeoIoDladFeatureColor,random.choice(colors)) )

## Polar Selection

In [35]:
f = go.FigureWidget(data=go.Scatterpolar(r = faults['dip'].to_list(), theta = faults['strike'].to_list(), text = faults['fault_name'].to_list(), mode='markers'))

In [36]:
def turn_on(trace, points, selector):
    fault_dlad.turn_off_items()
    fault_dlad.turn_on_items(faults.iloc[points.point_inds]['fault_name'].to_list())

In [37]:
data = f.data[0]

data.on_selection(turn_on)